<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Single-Record Ingestion for Time Series

This example walks through the Arize single-record API for ingesting time series data. Guides for all model types are available [here](https://docs.arize.com/arize/sending-data-to-arize/model-types).

## Install and Import Dependencies

In [ ]:
!pip install -q arize

import concurrent.futures as cf
import json
import urllib

from arize.api import Client
from arize.utils.types import ModelTypes, Environments, Schema

## Download and Display Data

In [ ]:
with urllib.request.urlopen(
    "https://storage.googleapis.com/arize-assets/documentation-sample-data/data-ingestion/time-series-assets/time_series_single_record.json?raw=true"
) as f:
    record = json.load(f)
print(json.dumps(record, indent=4))

## Create Arize Client

Sign up/ log in to your Arize account [here](https://app.arize.com/auth/login). Find your [space and API keys](https://docs.arize.com/arize/sending-data/sdk-reference/python-sdk/arize.init#retrieving-space-and-api-keys). Copy/paste into the cell below.

In [ ]:
SPACE_KEY = "SPACE_KEY"  # Change this line.
API_KEY = "API_KEY"  # Change this line.
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)
if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ CHANGE SPACE AND API KEYS")
else:
    print("✅ Arize client setup done! Now you can start using Arize!")

## Log Data to Arize

Log the record using the [single-record API](https://docs.arize.com/arize/sending-data-to-arize/data-ingestion-methods/sdk-reference/python-sdk/arize.log). In Arize, time series models are a subset of the [regression model type](https://docs.arize.com/arize/model-types/regression) and are characterized by three fields:
- The _forecast timestamp_ describes the date and time of the predicted event or observation and is passed into the timestamp field.
- The _run date_ describes the date on which the model was run and the prediction was generated and is optionally passed in as a tag.
- The _lag_ describes the number of days between the forecast timestamp and run date and is optionally passed in as a tag.

For example, if you run a model on Monday to predict the temperature on Friday, the run date would be Monday's date, the forecast timestamp would be a timestamp for a time on Friday and the lag would be four days.

In [ ]:
future = arize_client.log(
    prediction_id=record["prediction_id"],
    prediction_timestamp=record["forecast_ts"],
    features=record["features"],
    prediction_label=record["predicted_thermal"],
    actual_label=record["reported_thermal"],
    model_id="time-series-single-record-ingestion-tutorial",
    model_type=ModelTypes.REGRESSION,
    model_version="1.0.0",
    environment=Environments.PRODUCTION,
    tags=record["tags"],
)
result = future.result()

if result.status_code == 200:
    print(f"✅ Successfully logged data to Arize!")
else:
    print(
        f'❌ Logging failed with status code {result.status_code} and message "{result.text}"'
    )